In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, ConcatDataset

import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import json

In [2]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.losses import DiceLoss
from segmentation_models_pytorch.utils.metrics import IoU, Fscore, Accuracy

In [3]:
import matplotlib.pyplot as plt

# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image, 'gray')
    plt.show()

In [4]:
import random

SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

print(np.random.rand(5), torch.randn(5))

[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548 ] tensor([ 1.5410, -0.2934, -2.1788,  0.5684, -1.0845])


# Settings

In [5]:
root = os.getcwd()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

BATCH = 4

GEN_IN_CHANNELS = 1
GEN_N_CLASSES = 1

DIS_IN_CHANNELS = 1
DIS_N_CLASSES = 2

E_IN_CHANNELS = 1
E_N_CLASSES = 2

EPOCH = 200

### Model Settings

#### encoder

In [6]:
ENCODER = "resnet152"
ENCODER_WEIGHT = None

#### decoder

In [7]:
DECODER_ATT = "scse"

#### head

In [8]:
GEN_ACT = "sigmoid"
DIS_ACT = "softmax"
E_ACT = "softmax"

### Optimizer Settings

In [9]:
GEN_OPTIM_NAME = "adam"
GEN_init_lr = 1e-4
GEN_momentum = 0.9

In [10]:
DIS_OPTIM_NAME = "adam"
DIS_init_lr = 1e-3
DIS_momentum = 0.9
DIS_scheduler = "cosineAnnWarm"

In [11]:
E_OPTIM_NAME = "adam"
E_init_lr = 1e-3
E_momentum = 0.9
E_scheduler = "cosineAnnWarm"

# Epochs

In [12]:
from codes.losses import SSIMLoss
from codes.losses import MAELoss
from pytorch_msssim import ssim
from codes.metrics import PSNR, SNR, ContourEval
from codes.activation import Activation
from codes.utils import hu_clip_tensor
from codes.losses import PerceptualLoss
from kornia.filters.sobel import Sobel

In [13]:
def set_requires_grad(nets, requires_grad=False):
    """Set requies_grad=Fasle for all the networks to avoid unnecessary computations
    Parameters:
    nets (network list)   -- a list of networks
    requires_grad (bool)  -- whether the networks require gradients or not
    """
    if not isinstance(nets, list):
        nets = [nets]
        for net in nets:
            if net is not None:
                for param in net.parameters():
                    param.requires_grad = requires_grad

In [14]:
def replace_relu_to_leakyReLU(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            setattr(model, child_name, nn.LeakyReLU())
        else:
            replace_relu_to_leakyReLU(child)

In [15]:
def replace_bn_to_instanceNorm(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.BatchNorm2d):
            bn = getattr(model, child_name)
            setattr(model, child_name, nn.InstanceNorm2d(bn.num_features))
        else:
            replace_bn_to_instanceNorm(child)

In [16]:
perceptual_ext = PerceptualLoss()

In [17]:
sobel_filter = Sobel().to(device)

In [18]:
def train_a2b_epoch(epoch, generator, gen_optim, 
                    tf_discriminator, tf_d_optim, tf_scheduler, 
                    edge_discriminator, edge_d_optim, edge_scheduler, 
                    dataloader, device):

    # change mode to train and move to current device
    generator = generator.eval().to(device)
    tf_discriminator = tf_discriminator.train().to(device)
    edge_discriminator = edge_discriminator.train().to(device)
    
    tf_dis_l = 0
    edge_dis_l = 0
    tf_gen_l = 0
    edge_gen_l = 0

    set_requires_grad(tf_discriminator, True)
    set_requires_grad(edge_discriminator, True)
    set_requires_grad(generator, False)
    
    iteration = len(dataloader)
    
    for index, data in tqdm(enumerate(dataloader)):
        torch.cuda.empty_cache()
        
        x, y, air_x, bone_x, *_ = data

        x = x.to(device)
        y = y.to(device)
        air_x = air_x.to(device)
        bone_x = bone_x.to(device)

        y_pr = generator(x)
        y_pr = Activation(name=GEN_ACT)(y_pr) # zipped value to [0, 1]
        
        edge_x = sobel_filter(x)
        edge_y_pr = sobel_filter(y_pr)
        edge_y = sobel_filter(y)
        
       ############################
        # (1) Update True/Fake D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        tf_d_optim.zero_grad()

        y_cls = tf_discriminator(y)
        y_pr_cls = tf_discriminator(y_pr.detach())
        
        tf_dis = 0
        for _y_cls, _y_pr_cls in zip(y_cls, y_pr_cls):
            B, C = _y_cls.size()
            tf_dis = nn.CrossEntropyLoss()(_y_cls.float(), torch.zeros(B, dtype=torch.long, device=device)) + \
                            nn.CrossEntropyLoss()(_y_pr_cls.float(), torch.ones(B, dtype=torch.long, device=device)) + \
                            tf_dis

        tf_dis = tf_dis / (len(y_cls))
        tf_dis.backward()
        tf_d_optim.step()
        tf_scheduler.step(EPOCH + index / iteration)
        
        tf_dis_l += tf_dis.item()
        
       ############################
        # (1.1) Update Edge D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################        
        edge_d_optim.zero_grad()        
 
        edge_y_cls = edge_discriminator(edge_y)
        edge_pr_cls = edge_discriminator(edge_y_pr.detach())
        
        edge_dis = 0
        for _edge_y_cls, _edge_pr_cls in zip(edge_y_cls, edge_pr_cls):
            B, C = _edge_y_cls.size()
            edge_dis = nn.CrossEntropyLoss()(_edge_y_cls.float(), torch.zeros(B, dtype=torch.long, device=device)) + \
                                    nn.CrossEntropyLoss()(_edge_pr_cls.float(), torch.ones(B, dtype=torch.long, device=device)) + \
                                    edge_dis

        edge_dis = edge_dis / (len(edge_y_cls))
        edge_dis.backward()
        edge_d_optim.step()       
        edge_scheduler.step(EPOCH + index / iteration)
        
        edge_dis_l += edge_dis.item()

        
    ############################
    # (2) Update G network: maximize log(D(G(z)))
    ###########################
    
    # change mode to train and move to current device
    generator = generator.train().to(device)
    tf_discriminator = tf_discriminator.eval().to(device)
    edge_discriminator = edge_discriminator.eval().to(device)
    
    air_l = 0
    bone_l = 0
    mae_l = 0
    
    set_requires_grad(tf_discriminator, False)
    set_requires_grad(edge_discriminator, False)
    set_requires_grad(generator, True)
        
    for index, data in tqdm(enumerate(dataloader)):
        torch.cuda.empty_cache()
        
        x, y, air_x, bone_x, *_ = data

        x = x.to(device)
        y = y.to(device)
        air_x = air_x.to(device)
        bone_x = bone_x.to(device)

        y_pr = generator(x)
        y_pr = Activation(name=GEN_ACT)(y_pr) # zipped value to [0, 1]
        
        edge_x = sobel_filter(x)
        edge_y_pr = sobel_filter(y_pr)
        edge_y = sobel_filter(y)

        gen_optim.zero_grad()

        # adversarial loss
        assert y_pr.requires_grad, "ct_pred without gradient"
        assert edge_y_pr.requires_grad, "edge without gradient"
        
        y_pr_cls = tf_discriminator(y_pr)
        tf_gen = 0
        for _y_pr_cls in y_pr_cls:
            tf_gen = nn.CrossEntropyLoss()(_y_pr_cls.float(), torch.zeros(_y_pr_cls.size()[0], dtype=torch.long, device=device)) + tf_gen
        tf_gen = tf_gen / (len(y_pr_cls))
        
        edge_pr_cls = edge_discriminator(edge_y_pr)  
        edge_gen = 0
        for _edge_pr_cls in edge_pr_cls:
            edge_gen = nn.CrossEntropyLoss()(_edge_pr_cls.float(), torch.zeros(_edge_pr_cls.size()[0], dtype=torch.long, device=device)) + edge_gen
        edge_gen = edge_gen / (len(edge_pr_cls))

        
        # auxilary loss
        # https://discuss.pytorch.org/t/unclear-about-weighted-bce-loss/21486/2
        _min = -500
        _max = 500
        air_window = (-500, -499)
        upper = ((air_window[1]) - (_min))/(_max-(_min))
        lower = ((air_window[0]) - (_min))/(_max-(_min))
        air_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)
        air_loss = nn.BCELoss()(air_pr.float(), air_x.float())
        
        bone_window = (255, 256)
        upper = ((bone_window[1]) - (_min))/(_max-(_min))
        lower = ((bone_window[0]) - (_min))/(_max-(_min))
        bone_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)
        bone_loss = nn.BCELoss()(bone_pr.float(), bone_x.float())

        _loss = tf_gen + edge_gen + air_loss + bone_loss
        _loss.backward()    
        gen_optim.step()

        edge_gen_l += edge_gen.item()
        tf_gen_l += tf_gen.item()
        air_l += air_loss.item()
        bone_l += bone_loss.item()
        
    return  air_l/(index+1), bone_l/(index+1), \
                    edge_gen_l/(index+1), tf_gen_l/(index+1), \
                    edge_dis_l/(index+1), tf_dis_l/(index+1), \
                    tf_scheduler.get_last_lr()[0], edge_scheduler.get_last_lr()[0]

In [19]:
@torch.no_grad()
def eval_epoch(generator, tf_discriminator, edge_discriminator, dataloader, device):
  
    # change mode to train and move to current device
    generator = generator.eval().to(device)
    tf_discriminator = tf_discriminator.eval().to(device)
    edge_discriminator = edge_discriminator.eval().to(device)
    
    ssim_ = []
    psnr_ = []
    mae_ = []
    snr_ = []
    
    air_ = []
    bone_ = []
    cont_ = []
 
    tf_acc = []
    edge_acc = []
    
    for index, data in tqdm(enumerate(dataloader)):

        x, y, air_x, bone_x, *_ = data

        x = x.to(device)
        y = y.to(device)
        air_x = air_x.to(device)
        bone_x = bone_x.to(device)

        y_pr = generator(x)
        y_pr = Activation(name=GEN_ACT)(y_pr) # zipped value to [0, 1]
        
        edge_x = sobel_filter(x)
        edge_y_pr = sobel_filter(y_pr)
        edge_y = sobel_filter(y)
        
        total = x.size()[0]
       ############################
        # (1) Eval True/Fake D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################        
        y_cls = tf_discriminator(y)
        for _y_cls in y_cls:
            _y_cls = Activation(name=DIS_ACT)(_y_cls)
            _, _y_cls = torch.max(_y_cls.data, 1)
            correct = (_y_cls == torch.zeros(total, dtype=torch.long, device=device)).sum().item()
            tf_acc += [correct / total]        
        
        y_pr_cls = tf_discriminator(y_pr)
        for _y_pr_cls in y_pr_cls:
            _y_pr_cls = Activation(name=DIS_ACT)(_y_pr_cls)
            _, _y_pr_cls = torch.max(_y_pr_cls.data, 1)
            correct = (_y_pr_cls == torch.ones(total, dtype=torch.long, device=device)).sum().item()
            tf_acc += [correct / total]
        
       ############################
        # (1.1) Eval Edge D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################  
        edge_y_cls = edge_discriminator(edge_y)
        for _edge_y_cls in edge_y_cls:
            _edge_y_cls = Activation(name=DIS_ACT)(_edge_y_cls)
            _, _edge_y_cls = torch.max(_edge_y_cls.data, 1)
            correct = (_edge_y_cls == torch.zeros(total, dtype=torch.long, device=device)).sum().item()
            edge_acc += [correct / total]          
        
        edge_pr_cls = edge_discriminator(edge_y_pr)
        for _edge_pr_cls in edge_pr_cls:
            _edge_pr_cls = Activation(name=DIS_ACT)(_edge_pr_cls)
            _, _edge_pr_cls = torch.max(_edge_pr_cls.data, 1)
            correct = (_edge_pr_cls == torch.ones(total, dtype=torch.long, device=device)).sum().item()
            edge_acc += [correct / total]        
        
        ############################
        # (2) Eval G network: maximize log(D(G(z)))
        ###########################
        
        # main
        _ssim = ssim(y, y_pr, data_range=1.0, size_average=True) # return (N,)
        _psnr = PSNR()(y_pr, y, 1.0)
        _snr = SNR()(y_pr, y)
        _mae = MAELoss()(y_pr.float(), y.float())

        ssim_ += [_ssim.item()]
        psnr_ += [_psnr.item()]
        mae_ += [_mae.item()]
        snr_ += [_snr.item()]
        
        # auxilary
        _min = -500
        _max = 500
        air_window = (-500, -499)
        upper = ((air_window[1]) - (_min))/(_max-(_min))
        lower = ((air_window[0]) - (_min))/(_max-(_min))
        air_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)
        dice = Fscore()(air_pr, air_x)
        air_ += [dice.item()]
        
        bone_window = (255, 256)
        upper = ((bone_window[1]) - (_min))/(_max-(_min))
        lower = ((bone_window[0]) - (_min))/(_max-(_min))
        bone_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)
        dice = Fscore()(bone_pr, bone_x)
        bone_ += [dice.item()]

        for b in range(x.size()[0]):
            cont = ContourEval()(y_pr[b, :, :, :], x[b, :, :, :])
            cont_ += [cont.item()]

    return  sum(ssim_)/len(ssim_), sum(psnr_)/len(psnr_), sum(snr_)/len(snr_), sum(mae_)/len(mae_), \
                    sum(air_)/len(air_), sum(bone_)/len(bone_), sum(cont_)/len(cont_), sum(tf_acc)/len(tf_acc), sum(edge_acc)/len(edge_acc)

In [20]:
@torch.no_grad()
def test_epoch(iid, model, dataloader, device, save=False, path=None):
  
    # change mode to train and move to current device
    model = model.eval().to(device)
    
    for index, data in tqdm(enumerate(dataloader)):

        x, y, air_x, bone_x, *_ = data

        x = x.to(device)
        y = y.to(device)
        air_x = air_x.to(device)
        bone_x = bone_x.to(device)

        y_pr = model(x)
        y_pr = Activation(name=GEN_ACT)(y_pr) # zipped value to [0, 1]
        
        edge_x = sobel_filter(x)
        edge_y_pr = sobel_filter(y_pr)
        edge_y = sobel_filter(y)
        
        _min = -500
        _max = 500
        air_window = (-500, -499)
        upper = ((air_window[1]) - (_min))/(_max-(_min))
        lower = ((air_window[0]) - (_min))/(_max-(_min))
        air_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)
        
        bone_window = (255, 256)
        upper = ((bone_window[1]) - (_min))/(_max-(_min))
        lower = ((bone_window[0]) - (_min))/(_max-(_min))
        bone_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)

        for b in range(x.shape[0]):           
            tmp_y = y[b, :, :, :].unsqueeze(0)
            tmp_y_pr = y_pr[b, :, :, :].unsqueeze(0)
            tmp_air = air_x[b, :, :, :].unsqueeze(0)
            tmp_air_pr = air_pr[b, :, :, :].unsqueeze(0)
            tmp_bone = bone_x[b, :, :, :].unsqueeze(0)
            tmp_bone_pr = bone_pr[b, :, :, :].unsqueeze(0)
            
            _ssim = ssim(tmp_y, tmp_y_pr, data_range=1.0, size_average=True) # return (N,)
            _psnr = PSNR()(tmp_y_pr, tmp_y, 1.0)
            _snr = SNR()(tmp_y_pr, tmp_y)
            _mae = MAELoss()(tmp_y_pr.float(), tmp_y.float())
            _air = Fscore()(tmp_air_pr, tmp_air)
            _bone = Fscore()(tmp_bone_pr, tmp_bone)
            _cont = ContourEval()(tmp_y_pr, x[b, :, :, :])
            
            scores = {
                "ssim score": _ssim.item(),
                "psnr score": _psnr.item(),
                "snr score": _snr.item(),
                "mae error": _mae.item(),
                "air dice score": _air.item(),
                "bone dice score": _bone.item(),
                "contour dice score": _cont.item()
            }
            
            __cbct = (x.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __ct = (tmp_y.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __ct_pred = (tmp_y_pr.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __ct_masked = __ct_pred * tmp_air.squeeze().cpu().numpy()
            __air = (tmp_air.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __air_pr = (tmp_air_pr.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __bone = (tmp_bone.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __bone_pr = (tmp_bone_pr.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __edge_pr = (edge_y_pr[b, :].squeeze().cpu().numpy() * 255).astype(np.uint8)
            __edge_x = (edge_x.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __edge_y = (edge_y.squeeze().cpu().numpy() * 255).astype(np.uint8)
            
#             visualize(
#                 cbct = __cbct,
#                 ct_y = __ct,
#                 ct_pred = __ct_pred,
#                 edge_pred = __edge_pr,
#                 edge_x = __edge_x,
#                 edge_y = __edge_y,
#                 air = __air,
#                 air_pr = __air_pr,
#                 bone = __bone,
#                 bone_pr = __bone_pr
#             )
            
            if save:
                path_dir = os.path.join(path, "file_{}".format(iid))
                try:
                    os.mkdir(path_dir)
                except FileExistsError:
                    pass
                
                cv2.imwrite(os.path.join(path_dir, "cbct.jpg"), __cbct)
                cv2.imwrite(os.path.join(path_dir, "ct.jpg"), __ct)
                cv2.imwrite(os.path.join(path_dir, "ct_pred.jpg"), __ct_pred)
                cv2.imwrite(os.path.join(path_dir, "air.jpg"), __air)
                cv2.imwrite(os.path.join(path_dir, "air_pred.jpg"), __air_pr)
                cv2.imwrite(os.path.join(path_dir, "bone.jpg"), __bone)
                cv2.imwrite(os.path.join(path_dir, "bone_pred.jpg"), __bone_pr)
                cv2.imwrite(os.path.join(path_dir, "edge_pred.jpg"), __edge_pr)
                cv2.imwrite(os.path.join(path_dir, "edge_cbct.jpg"), __edge_x)
                cv2.imwrite(os.path.join(path_dir, "edge_ct.jpg"), __edge_y)
                with open(os.path.join(path_dir, "scores.txt"), "w") as file:
                    file.write(json.dumps(scores))                
            iid += 1

# Discriminator

In [21]:
from codes.activation import Activation
import torchvision.models as models
from codes.losses import MultiScaleHeads

In [22]:
class Discriminator(nn.Module):
    def __init__(self, in_channel=1, n_classes=2, activation=None):
        
        super(Discriminator, self).__init__()
        
        encoder = smp.Unet(encoder_name="resnet18", in_channels=in_channel, classes=n_classes).encoder
        self.encoder = encoder
        self.fc = MultiScaleHeads(n_classes=n_classes, channels=(512, ), activation=None)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.fc([x[-1]])
        return x

# Generator

In [23]:
import segmentation_models_pytorch as smp
from codes.decoder import UnetDecoder
from segmentation_models_pytorch.base.heads import SegmentationHead

In [24]:
class Generator(nn.Module):
    def __init__(self, encoder_name, encoder_weights, in_channels, classes, attention_type):
        
        super(Generator, self).__init__()
        
        self.encoder = smp.Unet(encoder_name=encoder_name, encoder_weights=encoder_weights, in_channels=in_channels, classes=classes).encoder
        self.decoder = UnetDecoder(
                            encoder_channels=[in_channels, 0, 0, 0, 0, 2048],
                            decoder_channels=[512, 256, 128, 64, 16],
                            n_blocks=5,
                            use_batchnorm=True,
                            attention_type=attention_type)
        self.head =  SegmentationHead(
                        in_channels=16,
                        out_channels=classes,
                        activation=None)
        
        
    def forward(self, x):
        x = self.encoder(x)[-1]
        x = self.decoder(*(None, None, None, None, x))
        x = self.head(x)
        
        return x

# Read Data

In [25]:
import glob
from codes.dataset import Dataset, DicomDataset, DicomsDataset
import codes.augmentation as aug

In [40]:
# run_name = wandb.run.name
run_name = "fluent-firebrand-185"

In [41]:
checkpoint = torch.load(os.path.join("weight-gan", "{}.pth".format(run_name)))

In [42]:
model = smp.Unet(encoder_name=ENCODER, 
                  encoder_weights=ENCODER_WEIGHT, 
                  in_channels=GEN_IN_CHANNELS, 
                  classes=GEN_N_CLASSES,
                 decoder_attention_type=DECODER_ATT)
replace_relu_to_leakyReLU(model)
replace_bn_to_instanceNorm(model)
model.load_state_dict(checkpoint["model"])

<All keys matched successfully>

In [43]:
tf_discriminator = Discriminator(in_channel=DIS_IN_CHANNELS, n_classes=DIS_N_CLASSES, activation=DIS_ACT)
replace_relu_to_leakyReLU(tf_discriminator)
replace_bn_to_instanceNorm(tf_discriminator)
tf_discriminator.load_state_dict(checkpoint["tf_discriminator"])

<All keys matched successfully>

In [44]:
edge_discriminator = Discriminator(in_channel=E_IN_CHANNELS, n_classes=E_N_CLASSES, activation=E_ACT)
replace_relu_to_leakyReLU(edge_discriminator)
replace_bn_to_instanceNorm(edge_discriminator)
edge_discriminator.load_state_dict(checkpoint["edge_discriminator"])

<All keys matched successfully>

# Pelvic

## Pelvic Test CBCT

In [45]:
test_case_path = 'raw/test/*_*'
paths = sorted(glob.glob(test_case_path))

In [46]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [47]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_cbct"))
except FileExistsError:
    pass

In [48]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], ditch=3, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=False)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "test_pelvic_cbct", patient_id)) 
    iid += len(scans)

28it [00:01, 19.31it/s]
27it [00:01, 19.59it/s]
26it [00:01, 19.13it/s]
28it [00:01, 19.51it/s]
27it [00:01, 19.75it/s]
28it [00:01, 19.70it/s]
27it [00:01, 19.63it/s]


In [49]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

191it [00:09, 19.86it/s]


(0.8376503761526177,
 23.28367394052875,
 14.287945852229733,
 0.027986927260279033,
 0.9949415224266396,
 0.879172175874427,
 0.3255242123341685,
 0.5,
 0.5)

## Pelvic Test CT

In [50]:
test_case_path = 'raw/test/*_*'
paths = sorted(glob.glob(test_case_path))

In [51]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [52]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_ct"))
except FileExistsError:
    pass

In [53]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], ditch=3, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=True)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "test_pelvic_ct", patient_id)) 
    iid += len(scans)

28it [00:01, 19.56it/s]
27it [00:01, 19.44it/s]
26it [00:01, 18.88it/s]
28it [00:01, 20.09it/s]
27it [00:01, 19.82it/s]
28it [00:01, 19.51it/s]
27it [00:01, 19.65it/s]


In [54]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

191it [00:09, 19.90it/s]


(0.9058759031495499,
 27.011169533454936,
 18.01544157497546,
 0.01823339551316662,
 0.994353358324591,
 0.8751763548776542,
 0.29369072101191074,
 0.4973821989528796,
 0.5)

## Pelvic L1 CBCT

In [55]:
test_case_path = 'L1_pelvic_processed/reg_pelvic_l1/*_*'
paths = sorted(glob.glob(test_case_path))

In [56]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [57]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_cbct"))
except FileExistsError:
    pass

In [58]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], ditch=3, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=False)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L1_pelvic_cbct", patient_id)) 
    iid += len(scans)

29it [00:01, 19.76it/s]
26it [00:01, 18.37it/s]
28it [00:01, 19.72it/s]
28it [00:01, 19.66it/s]
27it [00:01, 19.50it/s]
26it [00:01, 19.17it/s]
27it [00:01, 19.80it/s]
30it [00:01, 19.85it/s]
28it [00:01, 19.28it/s]
29it [00:01, 20.00it/s]
29it [00:01, 19.53it/s]
27it [00:01, 19.19it/s]
28it [00:01, 19.90it/s]
26it [00:01, 19.21it/s]
28it [00:01, 19.36it/s]
28it [00:01, 19.91it/s]
27it [00:01, 18.99it/s]
26it [00:01, 19.11it/s]
27it [00:01, 19.23it/s]
29it [00:01, 19.93it/s]
30it [00:01, 19.36it/s]
32it [00:01, 19.93it/s]
28it [00:01, 19.21it/s]
28it [00:01, 19.68it/s]
15it [00:00, 18.05it/s]
28it [00:01, 19.04it/s]
26it [00:01, 18.83it/s]
28it [00:01, 19.38it/s]
26it [00:01, 18.96it/s]
27it [00:01, 19.15it/s]
36it [00:01, 19.40it/s]
28it [00:01, 19.20it/s]
27it [00:01, 19.15it/s]
28it [00:01, 19.64it/s]
30it [00:01, 19.86it/s]
27it [00:01, 19.34it/s]
26it [00:01, 19.61it/s]
29it [00:01, 19.87it/s]
27it [00:01, 18.51it/s]
27it [00:01, 19.12it/s]
28it [00:01, 18.72it/s]
26it [00:01, 18.

In [59]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

1457it [01:12, 19.99it/s]


(0.8246404699618144,
 22.69954086610039,
 13.498523182640275,
 0.030303845382771758,
 0.9939522924016724,
 0.8444931893453381,
 0.3421277536952768,
 0.511324639670556,
 0.4742621825669183)

## Pelvic L1 CT

In [60]:
test_case_path = 'L1_pelvic_processed/reg_pelvic_l1/*_*'
paths = sorted(glob.glob(test_case_path))

In [61]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [62]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_ct"))
except FileExistsError:
    pass

In [63]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], ditch=3, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=True)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L1_pelvic_ct", patient_id)) 
    iid += len(scans)

29it [00:01, 18.98it/s]
26it [00:01, 18.67it/s]
28it [00:01, 19.43it/s]
28it [00:01, 18.80it/s]
27it [00:01, 19.80it/s]
26it [00:01, 19.04it/s]
27it [00:01, 18.92it/s]
30it [00:01, 19.65it/s]
28it [00:01, 19.43it/s]
29it [00:01, 19.91it/s]
29it [00:01, 19.31it/s]
27it [00:01, 18.85it/s]
28it [00:01, 19.10it/s]
26it [00:01, 18.92it/s]
28it [00:01, 19.61it/s]
28it [00:01, 19.16it/s]
27it [00:01, 19.48it/s]
26it [00:01, 19.04it/s]
27it [00:01, 19.05it/s]
29it [00:01, 19.41it/s]
30it [00:01, 19.25it/s]
32it [00:01, 19.82it/s]
28it [00:01, 19.19it/s]
28it [00:01, 19.53it/s]
15it [00:00, 17.41it/s]
28it [00:01, 19.78it/s]
26it [00:01, 18.74it/s]
28it [00:01, 19.44it/s]
26it [00:01, 19.46it/s]
27it [00:01, 19.36it/s]
36it [00:01, 18.52it/s]
28it [00:01, 18.97it/s]
27it [00:01, 18.62it/s]
28it [00:01, 19.48it/s]
30it [00:01, 19.74it/s]
27it [00:01, 19.41it/s]
26it [00:01, 18.81it/s]
29it [00:01, 19.32it/s]
27it [00:01, 18.91it/s]
27it [00:01, 19.21it/s]
28it [00:01, 19.37it/s]
26it [00:01, 19.

In [64]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

1457it [01:12, 20.18it/s]


(0.9027838986379771,
 27.087834874652557,
 17.886817182018564,
 0.01840480894116236,
 0.9943850749278296,
 0.8449658450552128,
 0.3375379529742648,
 0.4938229238160604,
 0.46705559368565547)

## Pelvic L2 CBCT

In [65]:
test_case_path = 'L2_pelvic_processed/reg_pelvic_l2/*_*'
paths = sorted(glob.glob(test_case_path))

In [66]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [67]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_cbct"))
except FileExistsError:
    pass

In [68]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], ditch=3, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=False)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L2_pelvic_cbct", patient_id)) 
    iid += len(scans)

26it [00:01, 19.04it/s]
26it [00:01, 17.44it/s]
26it [00:01, 19.10it/s]
27it [00:01, 18.92it/s]
31it [00:01, 19.53it/s]
27it [00:01, 18.07it/s]
27it [00:01, 18.66it/s]
26it [00:01, 18.52it/s]
28it [00:01, 18.24it/s]
28it [00:01, 19.31it/s]
27it [00:01, 19.00it/s]
27it [00:01, 19.42it/s]
26it [00:01, 18.86it/s]
26it [00:01, 18.42it/s]
26it [00:01, 18.74it/s]
26it [00:01, 18.54it/s]
26it [00:01, 18.72it/s]
26it [00:01, 18.40it/s]
27it [00:01, 18.69it/s]
26it [00:01, 17.78it/s]
26it [00:01, 18.51it/s]
28it [00:01, 18.38it/s]
28it [00:01, 18.77it/s]
29it [00:01, 18.62it/s]
28it [00:01, 19.24it/s]
26it [00:01, 18.90it/s]
27it [00:01, 17.90it/s]
26it [00:01, 18.63it/s]
28it [00:01, 18.74it/s]
30it [00:01, 18.04it/s]
27it [00:01, 18.15it/s]
29it [00:01, 19.23it/s]
27it [00:01, 18.85it/s]
27it [00:01, 18.86it/s]
26it [00:01, 18.50it/s]
26it [00:01, 18.57it/s]
26it [00:01, 19.25it/s]
26it [00:01, 18.15it/s]
27it [00:01, 18.61it/s]
26it [00:01, 17.96it/s]
26it [00:01, 18.90it/s]
26it [00:01, 18.

In [69]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

3064it [02:32, 20.06it/s]


(0.8124619180399818,
 22.065386078687624,
 12.480537968364304,
 0.03208064260636031,
 0.9935608989048761,
 0.8034307246844659,
 0.42440058485978455,
 0.5177872062663186,
 0.5016318537859008)

## Pelvic L2 CT

In [70]:
test_case_path = 'L2_pelvic_processed/reg_pelvic_l2/*_*'
paths = sorted(glob.glob(test_case_path))

In [71]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [72]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_ct"))
except FileExistsError:
    pass

In [73]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], ditch=3, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=True)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L2_pelvic_ct", patient_id)) 
    iid += len(scans)

26it [00:01, 18.23it/s]
26it [00:01, 16.87it/s]
26it [00:01, 18.53it/s]
27it [00:01, 18.45it/s]
31it [00:01, 18.51it/s]
27it [00:01, 18.08it/s]
27it [00:01, 18.05it/s]
26it [00:01, 18.24it/s]
28it [00:01, 18.20it/s]
28it [00:01, 18.72it/s]
27it [00:01, 18.23it/s]
27it [00:01, 18.15it/s]
26it [00:01, 17.86it/s]
26it [00:01, 18.37it/s]
26it [00:01, 18.22it/s]
26it [00:01, 18.70it/s]
26it [00:01, 18.45it/s]
26it [00:01, 18.20it/s]
27it [00:01, 18.09it/s]
26it [00:01, 17.82it/s]
26it [00:01, 18.29it/s]
28it [00:01, 18.27it/s]
28it [00:01, 18.36it/s]
29it [00:01, 17.98it/s]
28it [00:01, 18.87it/s]
26it [00:01, 18.69it/s]
27it [00:01, 17.96it/s]
26it [00:01, 17.70it/s]
28it [00:01, 18.11it/s]
30it [00:01, 18.22it/s]
27it [00:01, 17.91it/s]
29it [00:01, 18.68it/s]
27it [00:01, 17.72it/s]
27it [00:01, 18.12it/s]
26it [00:01, 17.92it/s]
26it [00:01, 17.80it/s]
26it [00:01, 18.89it/s]
26it [00:01, 18.11it/s]
27it [00:01, 18.16it/s]
26it [00:01, 17.82it/s]
26it [00:01, 18.46it/s]
26it [00:01, 18.

In [74]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

3064it [02:32, 20.07it/s]


(0.9065183435150911,
 27.515279837750267,
 17.93043167208256,
 0.016398019113580332,
 0.992451684462106,
 0.8406364873905453,
 0.3282089279491004,
 0.49592036553524804,
 0.49787859007832896)

# Abdomen

In [75]:
test_case_path = 'abdomen/test/*_*'
paths = sorted(glob.glob(test_case_path))

## Abdomen on CBCT

In [76]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [77]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_cbct"))
except FileExistsError:
    pass

In [78]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], ditch=3, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=False)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "abdomen_test_cbct", patient_id)) 
    iid += len(scans)

28it [00:01, 17.46it/s]
78it [00:04, 19.25it/s]
50it [00:02, 19.05it/s]
29it [00:01, 18.70it/s]
50it [00:02, 18.34it/s]
71it [00:03, 18.79it/s]
39it [00:02, 18.10it/s]


In [79]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

345it [00:18, 19.12it/s]


(0.7453349039174508,
 20.314491396364957,
 10.84513246011043,
 0.04377685960220254,
 0.9889411633293969,
 0.7090094755898803,
 0.29053558245085287,
 0.591304347826087,
 0.4434782608695652)

## Abdomen on CT

In [80]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [81]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_ct"))
except FileExistsError:
    pass

In [82]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], ditch=3, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=True)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "abdomen_test_ct", patient_id)) 
    iid += len(scans)

28it [00:01, 17.38it/s]
78it [00:04, 19.29it/s]
50it [00:02, 18.99it/s]
29it [00:01, 17.96it/s]
50it [00:02, 18.09it/s]
71it [00:03, 18.81it/s]
39it [00:02, 18.13it/s]


In [83]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

345it [00:18, 19.17it/s]


(0.8724188293235889,
 25.854804583563322,
 16.385445699829987,
 0.02129707207120415,
 0.9889179856764405,
 0.7142928537251565,
 0.297472535300514,
 0.5246376811594203,
 0.4579710144927536)

# Chest

In [84]:
test_case_path = 'chest/test/*_*'
paths = sorted(glob.glob(test_case_path))

## Chest on CBCT

In [85]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [86]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "chest_test_cbct"))
except FileExistsError:
    pass

In [87]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], ditch=3, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=False)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "chest_test_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "chest_test_cbct", patient_id)) 
    iid += len(scans)

49it [00:02, 18.56it/s]
39it [00:02, 18.12it/s]
40it [00:02, 18.30it/s]
29it [00:01, 16.34it/s]
35it [00:02, 17.06it/s]
37it [00:02, 17.06it/s]
34it [00:02, 16.96it/s]


In [88]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

263it [00:13, 19.05it/s]


(0.7586619004097275,
 20.11346147631511,
 9.665590861904304,
 0.03982503561324267,
 0.9493527678830612,
 0.761227679422513,
 0.431634324218478,
 0.5798479087452472,
 0.5209125475285171)

## Chest on CT

In [89]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [90]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "chest_test_ct"))
except FileExistsError:
    pass

In [91]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], ditch=3, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=True)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "chest_test_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "chest_test_ct", patient_id)) 
    iid += len(scans)

49it [00:02, 18.14it/s]
39it [00:02, 18.26it/s]
40it [00:02, 18.88it/s]
29it [00:01, 16.16it/s]
35it [00:02, 17.34it/s]
37it [00:02, 17.72it/s]
34it [00:01, 17.33it/s]


In [92]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

263it [00:13, 19.14it/s]


(0.8869273123179099,
 26.097605455057703,
 15.649734703760183,
 0.018305657314625304,
 0.9505951293234176,
 0.7298580403101286,
 0.4116745625168609,
 0.55893536121673,
 0.46387832699619774)

# Headneck

In [93]:
test_case_path = 'headneck/test/*_*'
paths = sorted(glob.glob(test_case_path))

## Headneck on CBCT

In [94]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [95]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "hn_test_cbct"))
except FileExistsError:
    pass

In [96]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], ditch=3, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=False)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "hn_test_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "hn_test_cbct", patient_id)) 
    iid += len(scans)

58it [00:02, 20.16it/s]
55it [00:02, 20.33it/s]
55it [00:02, 20.16it/s]
56it [00:02, 20.24it/s]
56it [00:02, 19.91it/s]
55it [00:02, 20.00it/s]
56it [00:02, 19.87it/s]


In [97]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

391it [00:19, 19.67it/s]


(0.7576800511621148,
 16.69703947003845,
 2.744486655954205,
 0.06620194238450978,
 0.5450454469130613,
 0.6896501359739787,
 0.026674327386999392,
 0.5,
 0.5217391304347826)

## Headneck on CT

In [98]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [99]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "hn_test_ct"))
except FileExistsError:
    pass

In [100]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], ditch=3, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=True)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "hn_test_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "hn_test_ct", patient_id)) 
    iid += len(scans)

58it [00:02, 19.75it/s]
55it [00:02, 19.89it/s]
55it [00:02, 19.63it/s]
56it [00:02, 19.82it/s]
56it [00:02, 19.86it/s]
55it [00:02, 19.51it/s]
56it [00:02, 19.76it/s]


In [101]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, identity=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

391it [00:19, 20.10it/s]


(0.7836413457418037,
 18.6116176883278,
 4.659064844412648,
 0.05795088744557956,
 0.5586336390184018,
 0.711433716382452,
 0.09511741669589259,
 0.49104859335038364,
 0.49872122762148335)